In [1]:
import pandas as pd
import sys
sys.path.append("/root/workSpace/investProject/ConvertBondWheel")

In [2]:
from policy.double_low_enhance.alarm import *

/root/workSpace/investProject/ConvertBondWheel


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO)

In [4]:
#对应配置 /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/config.py
double_low_conf = {
    'db_topk': 0.1,
    'min_curr_iss_amt': 9,  # <剩余规模
    'volatility': 0.1,  # >波动率
    'premium_rt': 0.2,  # <溢价率
    'min_price': 115,  # <可转债价格
    'min_dblow': 125,  # <双低价格
    'redeem': ['Y'],  #  Y=强赎 不在强赎
    'min_year_left': 1,  # >剩余到期年限 (不要一年内到期的)
    'ytm_rt': -0.01,  # > 到期收益率
    'pluse_out_step':10, # 脉冲轮出时， 轮出债比轮入新债双低值大多少
    'pulse_out_price_median':110, # 脉冲轮出时， 市场价格中位数
    'pulse_out_price':[120, 125], # 脉冲轮出时< 110 和>110 对应的轮出价格
    'pulse_out_dblow':[125, 130],  #同上， 双低值
    'user_zh_columns':[u'转债名称', u'双低', u'现价', u'溢价率', u'剩余规模', u'波动率', u'涨跌幅', u'成交额(万元)', u'换手率'],
} 

#user_zh_columns
# 展示字段， 可以继续添加，
# 如果报错， 说明没有爬取到该字段， 可以通过以下步骤来新增：
# 1. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/fields_config.py
# jisilu_dict 新增对应key-value, jisulu_used_columns 新增对应英文名
# 2. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/items.py
# 添加  该字段 = scrapy.Field()

### 周期轮动

In [5]:
# 读当前可转债数据
bond = read_data()
balance_names = read_balancing()

#轮动策略
in_names, out_names = periodWheel(bond, balance_names, double_low_conf)
print("轮入: %d支， 轮出: %d支" %(len(in_names), len(out_names)))

# 结果换成中文header & 按列排序
result = to_display(bond, balance_names, in_names, out_names, user_zh_columns=double_low_conf['user_zh_columns'])
result

INFO:root:dblow Low: 123.120000, dblow high: 125.010000, last db:122.050000


轮入: 13支， 轮出: 0支


/root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/alarm.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_bond['bond_nm'] = d_bond['bond_nm'].apply(lambda s: mark_bond(s, balance_names, in_names, out_names))


,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*文科转债,108.59,99.37,9.22%,9.50,2801.00%,0.41%,1811.88,192.00%,128127,文科园林,4.44,1.14%,0.92,4.88,90.98,AA-,3.42,6.34,48.10%,2021-03-01,26-08-19,5.19,4.01%,3.27%,X
1,*靖远转债,108.70,101.70,7.00%,28.00,4556.00%,0.48%,5379.94,189.00%,127027,靖远煤电,3.07,0.33%,0.83,3.23,95.05,AA+,2.26,4.20,39.90%,2021-06-16,26-12-09,5.50,2.36%,1.84%,X
2,*鸿达转债,109.09,106.89,2.20%,7.78,4838.00%,-1.94%,7481.76,885.00%,128085,鸿达兴业,4.10,-3.98%,1.42,3.92,104.59,B,2.74,5.10,6.40%,2020-06-22,25-12-16,4.52,3.49%,2.55%,X
4,*东湖转债,110.53,103.79,6.74%,15.50,3035.00%,-0.24%,2606.74,162.00%,110080,东湖高新,5.99,-0.50%,0.88,6.16,97.24,AA,4.31,8.01,33.40%,2021-10-18,27-04-12,5.84,1.82%,1.34%,X
5,*孚日转债,112.16,102.80,9.36%,6.50,3185.00%,0.31%,967.36,145.00%,128087,孚日股份,4.23,0.00%,1.06,4.50,94.00,AA,3.15,5.85,17.00%,2020-06-23,25-12-17,4.52,2.14%,1.60%,X
6,*利群转债,112.80,99.96,12.84%,18.00,2434.00%,0.00%,729.70,41.00%,113033,利群股份,6.21,-0.16%,1.10,7.01,88.59,AA,4.91,9.11,34.70%,2020-10-09,26-03-31,4.80,3.00%,2.42%,X
7,*中装转2,113.15,101.55,11.60%,11.60,3953.00%,0.14%,2596.22,220.00%,127033,中装建设,5.76,0.35%,1.23,6.33,91.00,AA,4.43,8.23,31.60%,2021-10-22,27-04-15,5.85,2.51%,1.98%,X
8,*嘉泽转债,113.31,106.19,7.12%,11.84,4946.00%,0.68%,1437.75,114.00%,113039,嘉泽新能,3.43,1.18%,1.96,3.46,99.13,AA,2.42,4.50,16.40%,2021-03-01,26-08-23,5.20,1.24%,0.78%,X
12,*海波转债,115.34,109.90,5.44%,2.45,4073.00%,0.27%,578.33,214.00%,123080,海波重科,12.07,2.03%,2.62,11.58,104.23,A+,8.11,15.05,18.00%,2021-06-08,26-12-01,5.48,2.66%,1.84%,X
13,*英特转债,116.76,108.66,8.10%,6.00,6548.00%,-0.87%,2764.44,421.00%,127028,英特集团,13.47,-2.04%,1.85,13.40,100.52,AA+,9.38,17.42,21.50%,2021-07-12,27-01-04,5.57,0.73%,0.30%,X


In [6]:
result.describe()

,双低,现价,剩余规模,成交额(万元),代码,正股价,PB,转股价,转股价值,强赎触发价,剩余年限
count,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00
mean,117.04,105.28,8.14,2880.45,120320.31,8.08,1.77,8.59,94.47,11.17,5.08
std,4.69,3.94,5.51,4255.14,6820.93,3.32,0.71,3.61,6.95,4.70,0.69
min,108.59,99.37,2.45,162.44,110071.00,3.07,0.83,3.23,81.61,4.20,2.73
25%,113.19,103.00,5.29,965.25,113568.50,5.82,1.25,6.23,89.65,8.10,4.76
50%,118.15,104.79,7.16,1449.38,123084.50,8.05,1.70,8.68,93.08,11.29,5.20
75%,119.83,106.87,8.95,2543.44,127031.75,10.08,2.09,10.58,98.89,13.75,5.50
max,124.24,116.53,28.00,20839.11,128127.00,14.63,3.79,16.98,110.44,22.07,5.88


### 脉冲轮动

In [7]:
# 立刻执行爬虫（集思录 & 雪球爬虫）
! sh /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/run_spider.sh

---------scrapy---------
2021-06-13 22:57:36 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: ConvertBondWheel)
2021-06-13 22:57:36 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 07:30:14) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.1.1, Platform Linux-3.10.0-862.el7.x86_64-x86_64-with-glibc2.10
2021-06-13 22:57:36 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'ConvertBondWheel',
 'CONCURRENT_REQUESTS': 64,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 64,
 'CONCURRENT_REQUESTS_PER_IP': 64,
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'ConvertBondWheel.spiders',
 'SPIDER_MODULES': ['ConvertBondWheel.spiders']}
2021-06-13 22:57:36 [scrapy.extensions.telnet] INFO: Telnet Password: 054e7c2a4c4877eb
2021-06-13 22:57:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetCons

In [8]:
# 轮动策略
bond = read_data()
balance_names = read_balancing()

in_names, out_names = pulseWheel(bond, balance_names, double_low_conf)
print("轮入: %d支， 轮出: %d支" %(len(in_names), len(out_names)))
result = to_display(bond, balance_names, in_names, out_names, user_zh_columns=double_low_conf['user_zh_columns'])
result

轮入: 0支， 轮出: 0支


/root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/alarm.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_bond['bond_nm'] = d_bond['bond_nm'].apply(lambda s: mark_bond(s, balance_names, in_names, out_names))


,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*文科转债,108.59,99.37,9.22%,9.50,2801.00%,0.41%,1811.88,192.00%,128127,文科园林,4.44,1.14%,0.92,4.88,90.98,AA-,3.42,6.34,48.10%,2021-03-01,26-08-19,5.19,4.01%,3.27%,X
1,*靖远转债,108.70,101.70,7.00%,28.00,4556.00%,0.48%,5379.94,189.00%,127027,靖远煤电,3.07,0.33%,0.83,3.23,95.05,AA+,2.26,4.20,39.90%,2021-06-16,26-12-09,5.50,2.36%,1.84%,X
2,*鸿达转债,109.09,106.89,2.20%,7.78,4838.00%,-1.94%,7481.76,885.00%,128085,鸿达兴业,4.10,-3.98%,1.42,3.92,104.59,B,2.74,5.10,6.40%,2020-06-22,25-12-16,4.52,3.49%,2.55%,X
4,*东湖转债,110.53,103.79,6.74%,15.50,3035.00%,-0.24%,2606.74,162.00%,110080,东湖高新,5.99,-0.50%,0.88,6.16,97.24,AA,4.31,8.01,33.40%,2021-10-18,27-04-12,5.84,1.82%,1.34%,X
5,*孚日转债,112.16,102.80,9.36%,6.50,3185.00%,0.31%,967.36,145.00%,128087,孚日股份,4.23,0.00%,1.06,4.50,94.00,AA,3.15,5.85,17.00%,2020-06-23,25-12-17,4.52,2.14%,1.60%,X
6,*利群转债,112.80,99.96,12.84%,18.00,2434.00%,0.00%,729.70,41.00%,113033,利群股份,6.21,-0.16%,1.10,7.01,88.59,AA,4.91,9.11,34.70%,2020-10-09,26-03-31,4.80,3.00%,2.42%,X
7,*中装转2,113.15,101.55,11.60%,11.60,3953.00%,0.14%,2596.22,220.00%,127033,中装建设,5.76,0.35%,1.23,6.33,91.00,AA,4.43,8.23,31.60%,2021-10-22,27-04-15,5.85,2.51%,1.98%,X
8,*嘉泽转债,113.31,106.19,7.12%,11.84,4946.00%,0.68%,1437.75,114.00%,113039,嘉泽新能,3.43,1.18%,1.96,3.46,99.13,AA,2.42,4.50,16.40%,2021-03-01,26-08-23,5.20,1.24%,0.78%,X
12,*海波转债,115.34,109.90,5.44%,2.45,4073.00%,0.27%,578.33,214.00%,123080,海波重科,12.07,2.03%,2.62,11.58,104.23,A+,8.11,15.05,18.00%,2021-06-08,26-12-01,5.48,2.66%,1.84%,X
13,*英特转债,116.76,108.66,8.10%,6.00,6548.00%,-0.87%,2764.44,421.00%,127028,英特集团,13.47,-2.04%,1.85,13.40,100.52,AA+,9.38,17.42,21.50%,2021-07-12,27-01-04,5.57,0.73%,0.30%,X
